# 2. 벡터 데이터베이스 구축

---

- 수집한 커리어넷 데이터(직업백과, 전공정보, 대학정보)를 Chroma 벡터 데이터베이스에 저장

# 1) 라이브러리 설치

In [1]:
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters "unstructured[md]" nltk tqdm

# 2) 스크래핑한 데이터 분할(Split)

In [2]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter


# 스크래핑한 데이터 : 직업백과, 전공(학과)정보, 대학정보
file_paths = [
    "../demo/result/CareerNet_CareerInfo.md",
    "../demo/result/CareerNet_MajorInfo.md",
    "../demo/result/CareerNet_UnivInfo.md"
]

# 파일 읽기
file_contents = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        file_contents.append(content)

# MarkdownHeaderTextSplitter를 사용하여 '마크다운 헤더'를 기준으로 청크로 분할
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, # 분할할 헤더 기준
    strip_headers=False, # 청크에 헤더 값 제거 X (청크에 헤더 값 포함시킴)
)

# 분할된 결과를 다시 RecursiveCharacterTextSplitter으로 분할
# 제목3의 내용이 하나의 청크에 전부 다 담기게 설정함. 
chunk_size = 700     # 분할할 청크의 크기
chunk_overlap = 120  # 분할할 청크 간의 중복 문자 수
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)    

# 분할
docs_list = []
for content in file_contents:
    md_header_splits = markdown_splitter.split_text(content)
    splits = text_splitter.split_documents(md_header_splits)
    docs_list.append(splits)

## Split 데이터 확인

---

In [13]:
print(type(docs_list))
print(type(docs_list[0]))    # 문서 1개 타입
print(type(docs_list[0][0])) # 청크 한개 타입

<class 'list'>
<class 'list'>
<class 'langchain_core.documents.base.Document'>

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [ ]:
# Split 결과 확인
dict = {0:'직업백과', 1:'전공정보', 2:'대학정보'}
for i, docs in enumerate(docs_list):
    print(f'==========["{dict[i]}" 데이터 ]===========')
    for header in docs[:3]:  # 각 리스트에서 처음 세 개의 항목만 선택
        print(f"{header.page_content}")
        print(f"{header.metadata}", end="\n------------------------\n")

---

# 3) 분할된 청크로 벡터 데이터베이스 구축

- 임베딩 모델 후보
    - ❎ intfloat/e5-small : 
    - ✅ BAAI/bge-m3
    - BM-K/KoSimCSE-roberta-multitask
    - intfloat/multilingual-e5-large-instruct

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from tqdm import tqdm


# 임베딩 모델 설정
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs = {'device': 'cpu'}, # default
    encode_kwargs = {'normalize_embeddings': False}, # default
)

persist_dir = "./chroma_db" # 벡터 DB 저장경로
batch_size = 1000  # 한 번에 처리할 배치 크기

# docs_list와 컬렉션 이름 매핑
collections = {
    "careernet_job_db": docs_list[0],   # 직업백과 데이터
    "careernet_major_db": docs_list[1], # 학과(전공) 데이터
    "careernet_univ_db": docs_list[2]   # 대학 데이터
}

/Users/ormor/root/DoWADO/Dowado-v2-Server/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# 각 컬렉션에 대해 벡터스토어 생성 및 임베딩 수행
for collection_name, docs in collections.items():
    for i in tqdm(range(0, len(docs), batch_size), desc=f"Embedding {collection_name}"):
        batch_docs = docs[i:i + batch_size]  # 배치로 문서 나누기

        # 벡터 스토어 생성 및 저장
        vectorstore = Chroma.from_documents(
            documents=batch_docs,
            collection_name=collection_name,
            embedding=hf_embeddings,
            persist_directory=persist_dir
        )

# 4) 쿼리 테스트

---
- Chroma 벡터 데이터베이스 불러온 후, Retrievr로 변환
- 리트리버 유사도 검색 방식에 따른 테스트 필요 (similarity, mmr, similarity_score_threshold)
    - similarity : 가장 유사한 문서 검색 (필터링 X)
        - 쿼리와 가장 유사한 벡터를 기준으로 문서 반환, Cosine similarity / Euclidean distance
    - mmr : 유사성 + 다양성 기반 검색
        - MMR(Maximal Marginal Relevance)방식은 쿼리에 대한 관련 항목을 검색할 때 검색된 문서의 중복 을 피하는 방법 중 하나임. 단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 쿼리에 대한 문서의 관련성 과 이미 선택된 문서들과의 차별성을 동시에 고려함. (다양성을 고려한 방식)
    - similarity_score_threshold : 가장 유사한 문서 검색, 유사도 점수 기준 보다 낮으면 필터링
        - 임계값을 적절히 설정함으로써 관련성이 낮은 문서를 필터링 하고, 질의와 가장 유사한 문서만 선별 / {"score_threshold": 0.8} 으로하면, 유사도 점수가 0.8 이상인 문서만 반환됨

- vectorstore.as_retriever 파라미터
    - search_type : 검색 알고리즘(검색방식)
    - search_kwargs: 추가 검색 옵션
        - k: 반환할 문서 수 (기본값: 4)
        - score_threshold: similarity_score_threshold 검색의 최소 유사도 임계값
        - fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
        - lambda_mult: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
        - filter: 문서 메타데이터 기반 필터링

- 참고
    - MMR (Maximal Marginal Relevance) 알고리즘으로 검색 결과의 다양성 조절 가능
    - 메타데이터 필터링으로 특정 조건의 문서만 검색 가능
    - tags 매개변수를 통해 검색기에 태그 추가 가능

- 주의사항
    - search_type과 search_kwargs의 적절한 조합 필요
    - MMR 사용 시 fetch_k와 k 값의 균형 조절 필요
    - score_threshold 설정 시 너무 높은 값은 검색 결과가 없을 수 있음
    - 필터 사용 시 데이터셋의 메타데이터 구조 정확히 파악 필요
    - lambda_mult 값이 0에 가까울수록 다양성이 높아지고, 1에 가까울수록 유사성이 높아짐

## [1] 직업백과

- 모바일앱개발자와 시스템소프트웨어개발자의 차이점은 무엇인가요?
- 시각디자이너의 취업 방법과 관련된 진로 준비는 어떻게 하나요?
- 소프트웨어 개발에 관심이 많은데 관련된 구체적인 직업에는 어떤게 있을까요?
- 교사가 되려면 필요한 어떤 기술이나 지식이 필요할까요?
- 향후 아나운서 직업 전망은 어떻게 될까요?
- 소프트웨어에 관심이 많은데, 개발은 적성에 안맞는거 같아요. 관련된 직업에는 어떤게 있을까요? (응용)
- 데이터 관련 직군으로 나아가려면 어떤 학과를 전공하는게 좋을까요? (응용)

> 단순히 정보를 묻는 질문에는 잘 대답(검색)하지만, 여러 데이터를 비교,분석해야하는 응용 질문에는 잘 대답(검색)하지 못함.

In [ ]:
# 직업백과 벡터 데이터베이스 불러오기
vectorstore = Chroma(
    embedding_function=hf_embeddings,  # 사용한 임베딩 모델과 동일한 함수
    persist_directory=persist_dir,     # 벡터 데이터베이스가 저장된 경로
    collection_name="careernet_job_db" # 불러올 컬렉션 이름
)

In [36]:
# 검색을 위한 retriever 설정
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
    },     
)

# 검색 테스트: 예시 쿼리로 유사 문서 검색
query = "교사가 되려면 필요한 어떤 기술이나 지식이 필요할까요?"  # 검색할 쿼리
test = retriever.invoke(query)

# 검색 결과 출력
for i, doc in enumerate(test):
    print(f"### 결과 {i + 1} ###")
    print(doc.page_content)  # 문서 내용 출력
    print(doc.metadata)      # 메타데이터 출력 (필요할 경우)
    print("\n")

### 결과 1 ###
일단 아까 말씀 드린 것처럼 제일 중요한 것은 내가 어떤 사람인지를 파악해야 될 것 같아요. 내가 어떤 것을 좋아하고 어떤 것을 싫어하고. 학생들이 적성이라고 하면 대부분 나중에 졸업하면 돈 많이 버나 이런 것이 적성이라고 생각하거든요. 말씀 드린 것처럼 가장 기본적인 것은 교사로서 내가 적성을 가지고 있는가 지금 아까 제가 다양한 진로를 가지고 있다라고 말씀은 드렸지만 그것은 변경이 가능하다는 것이지 기본적으로는 교사를 길러내는 곳이기 때문에 내가 일단 첫째는 교사로서 어떤 비전이나 적성을 가지고 있는가라는 것을 먼저 생각하고 있었으면 좋겠습니다. 또 그 다음에는 학생들이 대부분 면접 와서 얘기를 하면 그런 식으로 얘기를 해요. 나는 어려서부터 교사가 참 좋았었고 누가 좋았었고 이런 식으로 얘기를 해요. 그러면 면접관이 그 다음에 당연히 질문하는 것이 그러면 아무 데나 가지 왜 사회교육학과를 굳이 왔느냐고 하면 의외로 대답을 잘 못 해요. 심지어 이 일반 사회교육학과에서 뭘 가르치는지 조차도 모르는 경우도 있어요. 자기가 선택을 하고 싶어도 학교에서 선택을 안 해 줘서 못 듣는 수도 있거든요. 그렇다고 하면 학교에서는 그렇게 안 했지만 본인이 그것을 커버할 수 있도록 경험해 볼 수 있었으면 좋겠습니다. 어떤 캠프도 있을 것이고 책을 사다가 읽어보는 방법들도 있을 것이고 그런 식으로 해서 자기가 뭔가 의식적으로 노력을 해서 일단
{'Header 1': '사회교육과', 'Header 2': '인터뷰', 'Header 3': '내용:'}


### 결과 2 ###
> 질문 : 이 학과에 입학하면 가장 중요한 공부는 어떤 내용인가요?  
이 학과를 입학하는 학생들의 대부분은 중등학교 교사가 됩니다. 좋은 지구과학(과학) 교사가 되기 위해서는 전공 공부뿐만 아니라 교사로서 자질을 갖추는 것이 중요하다고 생각합니다. 가장 중요한 공부는 천문학, 대기과학, 지질학, 지구 물리학, 해양학과 같은 지구과학 내용과 관련된 기초적인 전공을 열심히 이수하는

## [2] 전공(학과)

- 00학과 졸업하면 어떤 직업으로 주로 진출할 수 있나요?
- 00학과로 진학하려면 고등학교 때 어떤 활동을 준비하는 게 도움이 되나요?
- 저는 무언가 만드는 것이 재밌고, 깊게 탐구하는 활동에 흥미가 많은데 어떤 전공을 선택하는게 좋을까요?
- 00전공의 졸업생들이 주로 어떤 분야로 진출하나요?
- 컴퓨터공학과에 진학하게 되면 배우게 되는 주요 전공과목에 대해서 알려줘.

In [24]:
# 기존 벡터 DB 불러오기
vectorstore = Chroma(
    embedding_function=hf_embeddings,  # 사용한 임베딩 모델과 동일한 함수
    persist_directory=persist_dir,     # 벡터 DB가 저장된 경로
    collection_name="careernet_major_db"     # 불러올 컬렉션 이름
)

# 검색을 위한 retriever 설정
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 유사도 검색 방식 "similarity" "mmr"
    search_kwargs={"k": 4}     # 상위 k개 유사 문서 반환
)

# 검색 테스트: 예시 쿼리로 유사 문서 검색
query = "저는 개발을 통해 프로그램을 만드는 것에 관심이 많고, 깊게 탐구하는 활동에 흥미가 많은데 어떤 전공을 선택하는게 좋을까요?"  # 검색할 쿼리
test = retriever.invoke(query)

# 검색 결과 출력
for i, doc in enumerate(test):
    print(f"### 결과 {i + 1} ###")
    print(doc.page_content)  # 문서 내용 출력
    print(doc.metadata)      # 메타데이터 출력 (필요할 경우)
    print("\n")

### 결과 1 ###
### 전공 관련 흥미와 적성  
컴퓨터, 게임, 스마트 워치, 각종 IT기술이 접목된 장치 등의 기능을 익히고 조작하는 것을 좋아하는 사람에게 유리합니다. 새로운 분야에 대한 호기심이 많고 다양한 기능을 가진 소프트웨어나 게임 등을 개발하는 일이므로 창의력과 논리력이 필요합니다.
{'Header 1': '응용소프트웨어공학과', 'Header 2': '전공 개요', 'Header 3': '전공 관련 흥미와 적성'}


### 결과 2 ###
### 내용:  
> 질문 : 교수님께서 이 학과(전공)를 선택하시게 된 동기는 무엇이었습니까?  
저는 서울대 컴퓨터공학과를 나왔는데 제 서울대 은사님이 그러시는 거에요. 우리나라가 소프트웨어를 수출할 수가 있을까, 우리가 미국에서 기술 받아서 수출하기 쉽지 않은 상황인데 옆에 일본을 보니까 게임이라고 하는 소프트웨어를 수출하더라, 우리가 게임이랑 소프트웨어를 만들면 수출이 가능할 수 있을 것 같다라는 말씀에서 저도 관심을 가지게 되었고, 실제로 지금 게임 수출을 엄청나게 많이 하고 있습니다.  
> 질문 : 어떤 분야에 관심을 갖고 있는 사람들이 이 학과에 입학을 하면 좋은가요?  
현재 우리 게임학과는 주력이 게임 개발입니다. 그래서 게임 개발에 관심이 있는 학생들이 들어오면 좋습니다. 게임이라고 하는 것이 융합 학문이다 보니까 스토리, 인문, 심리 이런 쪽에 관심이 있는 사람들도 필요하고 또 그림을 잘 그리는 예술가적인 사람, 컴퓨터 프로그래밍도 할 수 있는 공학적인 사람도 필요로 하는 융합 학문이라서 다양한 분야의 젊은이들이 들어올 수 있을 것이라고 생각합니다.  
> 질문 : 이 학과에서 입학하면 가장 중요한 공부는 어떤 내용인가요?
{'Header 1': '소프트웨어공학과', 'Header 2': '인터뷰', 'Header 3': '내용:'}


### 결과 3 ###
### 내용:  
> 질문 : 교수님께서 이 학과(전공)를 선택하시게 된 동기는 무엇이었습니까?  
우선은 제가 전공한 

## [3] 대학

In [17]:
# 기존 벡터 DB 불러오기
vectorstore = Chroma(
    embedding_function=hf_embeddings,  # 사용한 임베딩 모델과 동일한 함수
    persist_directory=persist_dir,     # 벡터 DB가 저장된 경로
    collection_name="careernet_univ_db"     # 불러올 컬렉션 이름
)

# 검색을 위한 retriever 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 유사도 검색 방식 "similarity" "mmr"
    search_kwargs={"k": 3}     # 상위 k개 유사 문서 반환
)

# 검색 테스트: 예시 쿼리로 유사 문서 검색
query = "UNIST"  # 검색할 쿼리
test = retriever.invoke(query)

# 검색 결과 출력
for i, doc in enumerate(test):
    print(f"### 결과 {i + 1} ###")
    print(doc.page_content)  # 문서 내용 출력
    print(doc.metadata)      # 메타데이터 출력 (필요할 경우)
    print("\n")

### 결과 1 ###
# 울산과학기술원(제1캠퍼스)  
- URL: http://www.unist.ac.kr/
- 학교종류: 대학(4년제)
- 학교유형: 대학교
- 설립: 국립
- 지역: 울산광역시  
---
{'Header 1': '울산과학기술원(제1캠퍼스)'}


### 결과 2 ###
# 유한대학교(제1캠퍼스)  
- URL: http://www.yuhan.ac.kr/
- 학교종류: 전문대학
- 학교유형: 전문대학
- 설립: 사립
- 지역: 경기도  
---
{'Header 1': '유한대학교(제1캠퍼스)'}


### 결과 3 ###
# 경남정보대학교(제1캠퍼스)  
- URL: http://www.kit.ac.kr/
- 학교종류: 전문대학
- 학교유형: 전문대학
- 설립: 사립
- 지역: 부산광역시  
---
{'Header 1': '경남정보대학교(제1캠퍼스)'}


